In [1]:
import json
import torch
import json
import os
from LayoutDM import CLDM
from torch.utils.data import DataLoader
from diffusers import DDPMScheduler
from PIL import Image
import numpy as np
from dataset import ImageLayout
import gc
from tqdm import tqdm

/workspace/joonsm/anaconda3/envs/CLDM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model with Seperate Param dict

In [2]:
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda:5


In [4]:
diffusion = DDPMScheduler(num_train_timesteps=250, prediction_type='sample', clip_sample=True)

In [3]:
def seg_loss(predicted_box, sample, zero_count):
    src = sample['sr']
    src_list = []
    
    for i in src:
        src_ = Image.open(i)
        src_list.append(src_)
    
    box = predicted_box.cpu().numpy()
    box = (box + 1) / 2
    
    match_list = []

    for i in range(box.shape[0]):
        img = src_list[i]
        width, height = img.size
        cx, cy, w, h = box[i]
        x = int((cx - w / 2) * width)
        y = int((cy - h / 2) * height)
        x2 = int((cx + w / 2) * width)
        y2 = int((cy + h / 2) * height)
        boxes = (x, y, x2, y2)
        
        # 이미지 크롭
        crop = img.crop(boxes)
        crop = np.array(crop)


        if crop.size == 0:
            print(f"Warning: Crop size is zero for box {boxes}.")
            match_list.append(0)  
            zero_count+=1
            continue
        
        blue_channel = crop[:, :, 2]
        blue_channel_flatten = blue_channel.flatten()
        

        match_pixel_size = np.sum(blue_channel_flatten == 128) / blue_channel_flatten.size
        match_list.append(match_pixel_size)
    
    # NaN 방지를 위해 match_list가 비어 있지 않은지 확인
    if len(match_list) == 0:
        print("Error: All crops have zero size. Returning NaN.")
        return float('nan')
    
    value = sum(match_list) / len(match_list)
    print(value)
    return value, zero_count

## Load Model with Seperate Param Dict

In [5]:
val = ImageLayout(type='val')
dataset = DataLoader(val, batch_size=256)

In [6]:
epoch =[50,100,150,200,250,300,350,400,450,500]

In [7]:
src={}
for index,value in enumerate(epoch):
    save_path = f"/workspace/joonsm/City_Layout/log_dir/FPN[18]_freeze/checkpoints/checkpoint-{value}/pytorch_model.bin"
    model = CLDM(use_temp=False,backbone_name='resnet18')
    model.load_state_dict(torch.load(save_path, map_location=device))
    model = model.to(device)
    model.eval()
    zero_count = 0
    batch_value = []
    step = 0
    with torch.no_grad():
        for step,batch in tqdm(enumerate(dataset), total=len(dataset)):
            shape = batch['box'].shape
            noisy_batch = {'image':batch['image'].to(device),
                    'box': torch.rand(*shape, dtype=torch.float32, device=device)}
            for i in range(250)[::-1]:
                t = torch.tensor([i]*shape[0], device=device)
                with torch.no_grad():
                    noise_pred = model(noisy_batch, timesteps=t)
                    bbox_pred = diffusion.step(noise_pred, t[0].detach().item(),  noisy_batch['box'], return_dict=True)
                    noisy_batch['box'] = bbox_pred.prev_sample
            predicted = bbox_pred.prev_sample
            value,zero_count = seg_loss(predicted, batch,zero_count)
            print(zero_count)
            batch_value.append(value)
            step +=1 

        final = sum(batch_value)/step
        src[index]={'score': final, 'zero_count':zero_count}
    del model
    gc.collect()

  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [01:42<13:42, 102.81s/it]

0.3104280299220361
3


 22%|██▏       | 2/9 [03:27<12:05, 103.69s/it]

0.24534482550987977
5


...

0.5099135689968932
0


 89%|████████▉ | 8/9 [41:46<05:14, 314.15s/it]

0.5458432240683982
2


100%|██████████| 9/9 [44:14<00:00, 294.92s/it]

0.2582441341849833
2


In [8]:
src

{0: {'score': 0.3093970914007919, 'zero_count': 9},
 1: {'score': 0.4192834340598356, 'zero_count': 0},
 2: {'score': 0.38265299153736554, 'zero_count': 0},
 3: {'score': 0.42349761716235085, 'zero_count': 11},
 4: {'score': 0.2207173885326622, 'zero_count': 3},
 5: {'score': 0.31368881680708, 'zero_count': 18},
 6: {'score': 0.4315178522563535, 'zero_count': 35},
 7: {'score': 0.544981215352927, 'zero_count': 10},
 8: {'score': 0.47554599710112705, 'zero_count': 0},
 9: {'score': 0.4979570349266622, 'zero_count': 2}}